In [1]:
import nltk
import numpy as np
import pandas as pd
import pickle
import string
from sklearn.preprocessing import OneHotEncoder


In [ ]:
print('a')

In [2]:
%matplotlib inline

In [3]:
pubmed = pickle.load( open( "./Pubmed/bins/pubmed_2012.p", "rb" )  )

In [4]:
pubmed.head(10)

,pubmedid,year,title,jounal,lang
86865,22251577,2012,Concentration fluctuations in binary fluid mem...,Journal of physics. Condensed matter : an Inst...,eng
86866,22251578,2012,The role of attractive many-body interaction i...,Journal of physics. Condensed matter : an Inst...,eng
86867,22251579,2012,Stochastic method for accommodation of equilib...,Journal of physics. Condensed matter : an Inst...,eng
86868,22251580,2012,Hidden octupole order in URu₂Si₂.,Journal of physics. Condensed matter : an Inst...,eng
86869,22251581,2012,Negative refraction via domain wall resonances...,Journal of physics. Condensed matter : an Inst...,eng
86870,22251582,2012,Complex phase behaviour from simple potentials.,Journal of physics. Condensed matter : an Inst...,eng
86871,22251583,2012,Interfaces and fluctuations in confined polyme...,Journal of physics. Condensed matter : an Inst...,eng
86872,22251584,2012,Ultraviolet photoinduced weak bonds in aryl-su...,Journal of physics. Condensed matter : an Inst...,eng
86873,22251585,2012,A simple physical description of DNA dynamics:...,Journal of physics. Condensed matter : an Inst...,eng
86874,22251586,2012,In situ XAFS and XRD studies of pressure-induc...,Journal of physics. Condensed matter : an Inst...,eng


In [8]:
values = dict()

In [9]:
for i, row in pubmed.iterrows():
    for c in row['title'].lower():
        if c not in values:
            values[c] =1
        

In [10]:
len(values)

315

In [11]:
values[' ']

1

In [12]:
# word to id dict
chr_to_id = {token: idx for idx, token in enumerate(values)}
id_to_chr = {idx: token for idx, token in enumerate(values)}


# convert token lists to token-id lists, e.g. [[1, 2], [2, 2]] here
#token_ids = [[word_to_id[token] for token in tokens_doc] for tokens_doc in tokens_docs]

In [13]:
len(chr_to_id)

315

In [14]:
# generate a continuous text woth all the titles. 
pubmed_corpus = list()
for i, row in pubmed.iterrows():
    pubmed_corpus.append(row['title'].lower())

pubmed_corpus = ' '.join(pubmed_corpus)

In [17]:
# float window 5 to 1
n_chars_corpus = len(pubmed_corpus)
max_len = 50
overlap = 5
dataX = []
dataY = []
for i in range(0, n_chars_corpus - max_len, overlap):
	seq_in = pubmed_corpus[i:i + max_len]
	seq_target = pubmed_corpus[i + max_len]
	dataX.append([chr_to_id[char] for char in seq_in])
	dataY.append(chr_to_id[seq_target])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns


Total Patterns:  18380182


In [18]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.datasets import imdb
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [19]:
X = np.zeros((n_patterns, max_len, len(chr_to_id)), dtype=np.bool)
y = np.zeros((n_patterns, len(chr_to_id)), dtype=np.bool)



# vectorize in LSTM format   
for idx,title in enumerate(dataX):
    for jdx, c in enumerate(title):
        X[idx, jdx, c] = 1
    y[idx, dataY[idx]] = 1
        
        
    
    

MemoryError: 

In [ ]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(256, input_shape=(max_len, len(chr_to_id))))
model.add(Dropout(0.2))
model.add(Dense(len(chr_to_id)))
model.add(Activation('softmax'))

#optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
print('-' * 50)
fileName= '2012_char'
filepath = fileName+"-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, y, batch_size=256, nb_epoch=20, callbacks=callbacks_list)

In [57]:
y.shape

(90000, 170717)

In [42]:
import random


In [44]:
 random.randint(0, len(word_to_id) )

14987

In [ ]:
for i in  random.randint(0, len(word_to_id) )

In [51]:
word_to_id['genetics']

37609

In [59]:
# my seed
seed = ['a','novel', 'study', 'on', 'human' ]

In [60]:

title_ids = return_ids(seed)



In [61]:
title_ids

[35340, 44721, 42855, 15283, 20841]

In [62]:
x = np.zeros((1, max_len, len(word_to_id)))
for t, word in enumerate(title_ids):
    x[0, t, word] =1 

In [66]:
pred = model.predict(x)[0]

In [68]:
pred.shape

(65659,)

In [69]:
temperature = 1
preds = np.asarray(pred).astype('float64')
preds = np.log(preds) / temperature
exp_preds = np.exp(preds)
preds = exp_preds / np.sum(exp_preds)
probas = np.random.multinomial(1, preds, 1)
np.argmax(probas)

38887

In [70]:
id_to_word[38887]

u'large'

In [71]:
seed2 = ['novel', 'study', 'on', 'human','large' ]
title_ids = return_ids(seed2)

x = np.zeros((1, max_len, len(word_to_id)))
for t, word in enumerate(title_ids):
    x[0, t, word] =1 
pred = model.predict(x)[0]
temperature = 1
preds = np.asarray(pred).astype('float64')
preds = np.log(preds) / temperature
exp_preds = np.exp(preds)
preds = exp_preds / np.sum(exp_preds)
probas = np.random.multinomial(1, preds, 1)
id_to_word[np.argmax(probas)]

u'heart'

In [ ]:
start_index = random.randint(0, len(raw_text) - max_len - 1)
#len(word_to_id)
import gc
for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    sentence = raw_text[start_index: start_index + maxlen]
    generated += ' '.join([value for value in sentence])
    print('----- Generating with seed: "' + ' '.join([value for value in sentence]) + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x = np.zeros((1, max_len, len(word_to_id)))
        for t, word in enumerate(sentence):
            x[0, t, char_indices[word]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char + " "
        sentence = sentence[1:]
        sentence.append(next_char)

        sys.stdout.write(" " + next_char)
        sys.stdout.flush()
    print()
gc.collect()